In [11]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
# Extract top n words and full words from each file. Used for a stratified vocab and for documents to feed into tfidf
def extract_words(filename, N):
    print(f"Processing {filename}...")
    df = pd.read_csv(filename)
    # fill in missing values with an empty string
    df = df.fillna("")
    top_words = [word for word, count in Counter(" ".join(df["snippet"]).lower().split()).most_common(N)]
    words = df["snippet"].tolist()
    label = df["language"].tolist()
    return words, top_words, label

#files = ["snippets-bash.csv", "snippets-c.csv", "snippets-cpp.csv", "snippets-csv.csv", 
#         "snippets-dotfile.csv", "snippets-go.csv", "snippets-html.csv", "snippets-java.csv", "snippets-javascript.csv",
#         "snippets-json.csv"]
files = ["snippets-all.csv"]

# Loosely based off number of reserved keywords per language. Trying to upper bound to track most common keywords 
# and other common symbols in each language 
# https://stackoverflow.com/questions/4980766/reserved-keywords-count-by-programming-language
N = 200

vocabulary = []
snippets = []
labels = []
for file in files:
    snippet_list, vocab, label = extract_words(file, N)
    vocabulary += vocab
    snippets += snippet_list
    labels += label

vocabulary = list(set(vocabulary))

Processing snippets-all.csv...


In [50]:
print(snippets[:5])
print(vocabulary[:5])

['test/files/normalise.jpg.png\ntest/files/normalise-resized.jpg\npackage-lock.json\n/package.json\n*.mongodb\n', 'a computer network, with no transfer of a copy, is not conveying.\n\n  An interactive user interface displays "Appropriate Legal Notices"\nto the extent that it includes a convenient and prominently visible\nfeature that (1) displays an appropriate copyright notice, and (2)\n', 'the predecessor has it or can get it with reasonable efforts.\n\n  You may not impose any further restrictions on the exercise of the\nrights granted or affirmed under this License.  For example, you may\nnot impose a license fee, royalty, or other charge for exercise of\n', '  THERE IS NO WARRANTY FOR THE PROGRAM, TO THE EXTENT PERMITTED BY\nAPPLICABLE LAW.  EXCEPT WHEN OTHERWISE STATED IN WRITING THE COPYRIGHT\nHOLDERS AND/OR OTHER PARTIES PROVIDE THE PROGRAM "AS IS" WITHOUT WARRANTY\nOF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO,\nTHE IMPLIED WARRANTIES OF MERCHANTABILI

In [51]:
vectorizer = TfidfVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(snippets)
# Lots of garbage in our features, e.g. '2', 'sherlock', '"",', 'carolina,south', '"仄仄平平仄，平平仄仄平"'
# Could be improved by creating smarter tokens, or engineering the vocabulary a bit more. 
print(vectorizer.get_feature_names())

['boolean', 'of', '2', 'new', '},', 'void', 'using', '|', '4', '],', '(', '-', 'only', 'which', 'and', 'name', 'static', 'use', 'you', '2,', '/>', 'function', 'private', '3', 'false', '//', 'break;', '=', 'then', 'can', 'license.', 'do', 'false,', 'end', 'is', 'one', 'case', 'license', '<div', 'should', 'def', 'test', 'msgstr', '});', 'let', 'any', 'public', 'null)', 'be', 'source', '#', 'resolved', 'int', '===', 'char', 'i', 'true;', '"author":', '"title":', 'true,', 'version', 'the', 'that', 'see', 'typedef', '&', 'has', 'copyright', 'must', '|0', 'for', '\\', 'c', '>', '@override', '*', '#:', '1;', 'default', 'not', '};', '""', 'x', 'distributed', 'error', '0x00,', ')', 'get', ':', 'a', 'when', '<<', '/**', 'an', '{', 'var', ':=', '"version":', 'on', 'list', 'nil', '[', '"type":', 'will', 'but', 'software', '}', '||', '1,', '#.', 'struct', 'code', 'if', '0,', 'true', 'else', 'object', 'at', '@param', '()', 'return', '0)', ');', ',', 'err', 'in', 'export', 'null;', 'bool', '0', '#def

In [55]:
languages = list(set(labels))
encoding = { language: number for language, number in zip(languages, range(len(languages)))}
# One-hot encode the language
y = pd.get_dummies(map(lambda x: encoding[x], labels))
print(X)
print(y)

print(X.shape[0])
print(len(y))

  (0, 41)	1.0
  (1, 183)	0.16341492692682041
  (1, 178)	0.2540222823849807
  (1, 143)	0.2746496819198951
  (1, 138)	0.22133967429620074
  (1, 136)	0.21002905477423364
  (1, 93)	0.45693483173087457
  (1, 79)	0.20601576540346922
  (1, 67)	0.2605086464445217
  (1, 62)	0.44328038094254124
  (1, 61)	0.1448822384138086
  (1, 34)	0.17770093565570846
  (1, 14)	0.37632548332860516
  (1, 1)	0.1744958356981105
  (2, 168)	0.40934888369881856
  (2, 165)	0.11086403613928053
  (2, 155)	0.16676641418483068
  (2, 138)	0.28709925810907
  (2, 136)	0.1362141378376166
  (2, 131)	0.34535837350404613
  (2, 98)	0.14513948997563972
  (2, 87)	0.15516635851368693
  (2, 79)	0.2672226458911143
  (2, 70)	0.22386171309966488
  (2, 66)	0.17987030893528658
  :	:
  (4849923, 162)	0.9081163723807725
  (4849923, 79)	0.41871786947058554
  (4849925, 152)	0.24672147556909907
  (4849925, 79)	0.9690864324161217
  (4849926, 152)	0.24672147556909907
  (4849926, 79)	0.9690864324161217
  (4849928, 162)	1.0
  (4849931, 152)	0.2467

In [53]:
# Save data in csv files
pd.DataFrame(X).to_csv("X.csv")
y.to_csv("y.csv")